# Creating a DataFrame

In [1]:
df = spark.createDataFrame([('Alice', 13), ('Bob', 12)], ['name', 'age'])
print(df.collect())

[Row(name='Alice', age=13), Row(name='Bob', age=12)]


# Loading Data

Spark supports various file formats. For example JSON.

In [3]:
persons = spark.read.json("s3://dimajix-training/data/persons.json")
persons

DataFrame[age: bigint, height: bigint, name: string, sex: string]

In [4]:
persons.collect()

[Row(age=14, height=156, name='Alice', sex='female'),
 Row(age=21, height=181, name='Bob', sex='male'),
 Row(age=27, height=176, name='Charlie', sex='male'),
 Row(age=24, height=167, name='Eve', sex='female'),
 Row(age=19, height=172, name='Frances', sex='female'),
 Row(age=31, height=191, name='George', sex='male')]

# Inspecting a DataFrame

Spark supports various methods for inspecting both the contents and the schema of a DataFrame

In [5]:
persons.show()

+---+------+-------+------+
|age|height|   name|   sex|
+---+------+-------+------+
| 14|   156|  Alice|female|
| 21|   181|    Bob|  male|
| 27|   176|Charlie|  male|
| 24|   167|    Eve|female|
| 19|   172|Frances|female|
| 31|   191| George|  male|
+---+------+-------+------+



In [6]:
persons.printSchema()

root
 |-- age: long (nullable = true)
 |-- height: long (nullable = true)
 |-- name: string (nullable = true)
 |-- sex: string (nullable = true)



## Pandas Interoperability

Spark also supports interoperation with Python Pandas, the standard library for modelling tabular data.

In [7]:
persons.toPandas()

,age,height,name,sex
0,14,156,Alice,female
1,21,181,Bob,male
2,27,176,Charlie,male
3,24,167,Eve,female
4,19,172,Frances,female
5,31,191,George,male


# Loading CSV Data

Of course Spark also supports reading CSV data. CSV files may optionally contain a header containing the column names.

In [8]:
persons = spark.read \
    .option("header","true") \
    .csv("s3://dimajix-training/data/persons_header.csv")
persons.toPandas()

,age,height,name,sex
0,23,156,Alice,female
1,21,181,Bob,male
2,27,176,Charlie,male
3,24,167,Eve,female
4,19,172,Frances,female
5,31,191,George,female


# Working with Columns

DataFrames are very similar to an SQL table. You can work with individual columns, transform them and create new columns.

In [23]:
from pyspark.sql.functions import *

result = persons.select('name', persons.age, col('height'), persons['sex'])
result.toPandas()

,name,age,height,sex
0,Alice,23,156,female
1,Bob,21,181,male
2,Charlie,27,176,male
3,Eve,24,167,female
4,Frances,19,172,female
5,George,31,191,female


In [10]:
# Result should be "Alice is 23 years old"
result = persons.select(
            concat(persons.name, lit(" is "), persons.age, lit(" years old")).alias("description")
        )
result.toPandas()

,description
0,Alice is 23 years old
1,Bob is 21 years old
2,Charlie is 27 years old
3,Eve is 24 years old
4,Frances is 19 years old
5,George is 31 years old


In [11]:
result = persons.select((persons.age * 2).alias("age2"))
result.toPandas()

,age2
0,46.0
1,42.0
2,54.0
3,48.0
4,38.0
5,62.0


# Adding and Removing Columns

Often it is very useful to add new columns as derived data. And at the same time, removing columns can also be useful in order to focus on relevant information. This is comparable to temporary tables in SQL.

In [12]:
result = persons.withColumn("even_odd_age", 
                when(persons.age % 2 == 0, "even").otherwise("odd")
        )
result.toPandas()

,age,height,name,sex,even_odd_age
0,23,156,Alice,female,odd
1,21,181,Bob,male,odd
2,27,176,Charlie,male,odd
3,24,167,Eve,female,even
4,19,172,Frances,female,odd
5,31,191,George,female,odd


In [13]:
result = persons.drop("sex")
result.toPandas()

,age,height,name
0,23,156,Alice
1,21,181,Bob
2,27,176,Charlie
3,24,167,Eve
4,19,172,Frances
5,31,191,George


# Filtering Data

Filtering corresponds to an SQL `WHERE` clause. Spark supports different equivalent variants for removing rows from a DataFrame.

In [14]:
result = persons.filter(persons.age > 22)
result.toPandas()

,age,height,name,sex
0,23,156,Alice,female
1,27,176,Charlie,male
2,24,167,Eve,female
3,31,191,George,female


In [15]:
result = persons.where((persons.age > 22) & (persons.height > 160))
result.toPandas()

,age,height,name,sex
0,27,176,Charlie,male
1,24,167,Eve,female
2,31,191,George,female


In [16]:
result = persons.filter("age > 22 AND height > 160")
result.toPandas()

,age,height,name,sex
0,27,176,Charlie,male
1,24,167,Eve,female
2,31,191,George,female


# Simple Aggregations

In [18]:
persons.count()

6

In [17]:
result = persons.select(
        max(persons.age).alias("max_age"), 
        avg(persons.height).alias("avg_height")
    )
result.toPandas()

,max_age,avg_height
0,31,173.833333


# Grouping and Aggregating

In [19]:
result = persons.groupBy(persons.sex).agg(
    avg(persons.age).alias("avg_age"),
    min(persons.height).alias("min_height"),
    max(persons.height).alias("max_height")
)
result.toPandas()

,sex,avg_age,min_height,max_height
0,female,24.25,156,191
1,male,24.00,176,181


# Sorting DataFrames

You can sort the entries (= rows) of a DataFrame by an arbitrary column or expression.

In [21]:
result = persons.sort(persons.height)
result.toPandas()

,age,height,name,sex
0,23,156,Alice,female
1,24,167,Eve,female
2,19,172,Frances,female
3,27,176,Charlie,male
4,21,181,Bob,male
5,31,191,George,female


In [22]:
result = persons.orderBy(persons.height.desc())
result.toPandas()

,age,height,name,sex
0,31,191,George,female
1,21,181,Bob,male
2,27,176,Charlie,male
3,19,172,Frances,female
4,24,167,Eve,female
5,23,156,Alice,female


# User Defined Functions

Sometimes the built in functions do not suffice or you want to call an existing function of a Python library. Using User Defined Functions (UDF) it is possible to wrap an existing function into a Spark DataFrame function.

In [20]:
import html
from pyspark.sql.types import *

html_encode = udf(html.escape, StringType())

df = spark.createDataFrame([
        ("Alice & Bob",),
        ("Thelma & Louise",)
    ], ["name"])

result = df.select(html_encode(df.name).alias("html_name"))
result.toPandas()

,html_name
0,Alice &amp; Bob
1,Thelma &amp; Louise
